In [1]:
import paddle.fluid as fluid
import numpy as np

from paddle.fluid.debugger import draw_block_graphviz

In [2]:
# 一个简单的网络
def network(inp):
    y = fluid.layers.fc(inp, size=1, act=None)
    return y

In [3]:
# 训练数据, y = 2x + 3
data_x = np.arange(128).reshape(128, 1).astype('float32')
data_y = (data_x * 2 + 3).reshape(128, 1)
test_x =np.array([1024, 600, 666]).reshape(3, 1).astype('float32')

In [4]:
main = fluid.Program()
startup = fluid.Program()
scope = fluid.Scope()

with fluid.unique_name.guard():
    with fluid.scope_guard(scope):
        with fluid.program_guard(main ,startup):
            
            # 预测网络
            x = fluid.layers.data('x', [1])
            predict_y = network(x)
        
            inference_program = main.clone(for_test=True)
        
            # 损失
            y = fluid.layers.data('y', [1])
            loss = fluid.layers.square_error_cost(input=predict_y, label=y)
            loss = fluid.layers.mean(loss)
            
            # 优化器
            opt = fluid.optimizer.Adam(learning_rate=0.1)
            opt.minimize(loss)

            # 初始化变量
            exe = fluid.executor.Executor(fluid.CPUPlace())
            exe.run(startup)

            # 训练
            for i in range(1000):
                result = exe.run(fetch_list=[loss], feed={
                    'x': data_x,
                    'y': data_y
                })
                if i % 100 == 0:
                    print("Epoch %i loss %f" %(i, result[0][0]))
            
            # 预测
            result = exe.run(inference_program, fetch_list=[predict_y], feed={
                'x': test_x
            })
            print(result)
            
            fluid.io.save_inference_model("models/linear", 
                                          feeded_var_names=['x'], 
                                          target_vars=[predict_y],
                                          executor=exe)

Epoch 0 loss 27859.640625
Epoch 100 loss 0.883998
Epoch 200 loss 0.115282
Epoch 300 loss 0.094322
Epoch 400 loss 0.073891
Epoch 500 loss 0.055575
Epoch 600 loss 0.040188
Epoch 700 loss 0.027956
Epoch 800 loss 0.018710
Epoch 900 loss 0.012042
[array([[2052.9038],
       [1204.0444],
       [1336.1782]], dtype=float32)]


In [5]:
exe = fluid.executor.Executor(fluid.CPUPlace())
inference_program, feed_names, fetch_list = fluid.io.load_inference_model("models/linear", exe)

In [6]:
draw_block_graphviz(inference_program.blocks[0], path="infer.pdf")

In [7]:
exe.run(inference_program, fetch_list=fetch_list, feed={
    'x': test_x
})

[array([[2052.9038],
        [1204.0444],
        [1336.1782]], dtype=float32)]